<a href="https://colab.research.google.com/github/juatc83/publicCodes/blob/main/TD3_Notre_premier_r%C3%A9seau_profond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch a une librairie de modèles et outils pour la vision : TorchVision https://pytorch.org/docs/stable/torchvision/index.html (téléchargeable ici : https://github.com/pytorch/vision, mais vous n'avez pas besoin de le télécharger pour ce TD car il est déjà installé dans Colab).

Parmi les modèles disponibles (https://pytorch.org/docs/stable/torchvision/models.html), nous allons utiliser AlexNet que nous avons découvert lors du dernier cours. TorchVision implémente une version d'AlexNet et a même une version du réseau déjà entrainée sur ImageNet. Nous allons charger ce modèle :

In [ ]:
import torch
from torchvision import models

model = models.alexnet(pretrained=True, progress=True)

**Exercice  0**

Vérifiez que l'architecture d'AlexNet est bien implémentée :

In [ ]:
print(model)

**Exercice 1**

*Chargement d'une image*

Nous allons tout d'abord nous exercer à utiliser un modèle déjà entrainé.

Vous trouverez une image de poisson clown sur Moodle. Téléchargez là et charger là dans Colab en utilisant le bouton 'importer' de l'onglet 'Fichiers' dans le volet de gauche de cette page.

TorchVision utilise Pillow par défaut pour manipuler les images. Nous allons donc charger l'image à l'aide de Pillow :

In [ ]:
from PIL import Image
img = Image.open("clown_fish.jpg")

from IPython.display import display
display(img)

*Préparation de l'image*

Puisque le modèle AlexNet a été pré-entrainé sur ImageNet, il s'attend à recevoir des images d'une certaine taille, et ayant été normalisées de la même manière que les images d'entrainement (voir les explications du pré-entrainement sur https://pytorch.org/docs/stable/torchvision/models.html).

TorchVision a justement un module de préparation d'image appelé `transforms` (https://pytorch.org/docs/stable/torchvision/transforms.html) que nous allons pouvoir utiliser pour rendre notre image adéquate.

Vérifiez sur https://pytorch.org/docs/stable/torchvision/transforms.html#transforms-on-pil-image-and-torch-tensor ce que font les différentes opérations de préparation implémentées ci-dessous.

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose([
 transforms.Resize(256),             
 transforms.CenterCrop(224),         
 transforms.ToTensor(),              
 transforms.Normalize(               
 mean=[0.485, 0.456, 0.406],         
 std=[0.229, 0.224, 0.225]           
 )])

# application de la transformation à notre image
img_t = preprocess(img)

# une image Tensor peut être visualisée avec Matplotlib
# en permutant les dimension (C, H, W) -> (H, W, C)
import matplotlib.pyplot as plt
plt.imshow(  img_t.permute(1, 2, 0)  )

Comme le modèle AlexNet s'attend à recevoir un batch d'images, on crée un batch d'une seule image :

In [ ]:
batch_t = torch.unsqueeze(img_t, 0)

*Evaluation de l'image par le modèle*

L'image est maintenant prête à être donnée au modèle pour estimer sa classification :

In [ ]:
# bascule du modèle en mode 'évaluation', c.a.d. 'test'
model.eval()

# déplace le modèle et le batch dans le GPU
if torch.cuda.is_available():
    batch_t = batch_t.to('cuda')
    model.to('cuda')

# inférence
with torch.no_grad():
  out = model(batch_t)

print(out[0].shape)

# normalisation des scores par softmax pour avoir une pseudo-probabilité
scores = torch.nn.functional.softmax(out[0], dim=0)
print(scores.sum())

Quelle est la taille de la sortie du réseau ? Que cela veut-il dire à votre avis ? (scrollez pour continuer, mais attention, spoiler !)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

Pour interpréter la réponse du réseau, nous devons au préalable charger les noms des classes correspondants aux éléments du vecteur de sortie :

In [ ]:
import requests
import json

LABELS_URL = 'https://s3.amazonaws.com/mlpipes/pytorch-quick-start/labels.json'
labels = {int(key):value for (key, value)
 in requests.get(LABELS_URL).json().items()}

En vous aidant des fonctions `torch.max` et `torch.sort`, déterminez la classe prédite par le modèle, et les 5 classes les plus probables. Affichez aussi leurs scores.

In [ ]:
## à compléter

Solution plus bas.

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

Solution :

In [ ]:
# classe avec le plus haut score
score, index = torch.max(out, 1)
print(index.item(), labels[index.item()], score.item())

# 5 meilleures classes
scores, indices = torch.sort(out, descending=True)
top5 = [(ind.item(), labels[ind.item()], sc.item()) for sc, ind in zip(scores[0][:5], indices[0][:5])]
print(top5)

Vous pouvez si vous le souhaitez essayer le modèle avec d'autres images à charger sur Colab.

**Exercice 2**

Nous allons maintenant entrainer notre propre modèle. Imaginons que nos ressources sont limitées, aussi bien en nombre d'images dans notre dataset qu'en temps de calcul. Il existe une solution très populaire à ce problème : le **pré-entrainement**.

Il est possible de charger un modèle pré-entrainé sur un grand dataset, puis de continuer son entrainement pour le spécialiser sur nos données et notre tâche. On appelle cela le **fine-tuning**.

Pendant le fine-tuning, les paramètres des premières couches bas-niveau d'un CNN n'auront pas besoin d'être réappris de manière significative car les filtres bas-niveaux sont probablement aussi bons pour nos images qu'ils l'étaient pour celles du dataset de pré-entrainement. Il faudra donc surtout ré-entrainer les paramètres des couches haut-niveau, ce qui demande moins d'itérations. De plus, ces couches étant initialisées avec des valeurs pas franchement aléatoires et probablement pas si bêtes que ça, nous aurons besoin d'encore moins d'itérations. Autre avantage du fine-tuning : le modèle a déjà appris à traiter une certaine variété de données, ce qui réduit les problèmes liés à la petite taille de notre dataset.

Dans notre cas, nous allons utiliser le modèle AlexNet pré-entrainé sur ImageNet, et nous le spécialiserons sur la classification des chiffres de MNIST.

Nous allons supposer que les descripteurs appris pour les images de ImageNet feront bien l'affaire pour les images de MNIST. Nous allons donc simplifier au maximum le fine-tuning pour ne changer que la classification d'AlexNet, en conservant donc l'extraction de features telle qu'elle est maintenant. En fait, nous n'allons même changer que la dernière couche de classification, celle qui renvoie les scores des classes. Cette version extrême du fine-tuning s'appelle **feature extraction** car nous utilisons les features du CNN et nous réapprenons simplement un nouveau classifier qui les utilise.

*Modification de l'architecture du réseau*

Nous avons examiné la structure d'AlexNet en début de ce TD. Nous avons vu que la dernière couche du réseau est la 6ème couche de classification :
`(6): Linear(in_features=4096, out_features=1000, bias=True)`

Comme MNIST a 10 classes et non 1000, il faut donc remplacer cette couche par une couche avec 10 sorties. **Complétez le code ci-dessous.**

Autre modification nécessaire : il faut dire à PyTorch de ne pas appliquer l'autograd aux couches qui ne doivent pas être modifiées pendant le fine-tuning (puisque nous avons décidé de ne modifier que la dernière couche).
Remarque : par défaut les couches nouvellement ajoutées ont `requires_grad = True`.

In [ ]:
for param in model.parameters():
  param.requires_grad = False

# modification de la dernière couche du réseau (à compléter)
model.classifier[6] = ### à compléter
print(model)

*Chargement du dataset*

TorchVision contient des fonctions d'aide pour télécharger les datasets les plus populaires (https://pytorch.org/docs/stable/torchvision/datasets.html). Nous pouvons donc facilement télécharger MNIST.

Puisque nous ne changeons pas les couches d'extraction de features du réseau, nous devons nous assurer que les images respectent la taille et la normalisation des images attendues par le réseau. Cela peut être fait automatiquement en passant un objet `transforms` de préparation des images à la fonction de chargement du dataset.

De plus, lors de l'entrainement, nous pouvons augmenter artificiellement la taille de notre set d'entrainement en appliquant des transformations de manière aléatoire à nos images. Cela s'appelle l'**augmentation**, et peut être aussi réalisé par l'objet `transforms`. **Examinez les transformations disponibles** (https://pytorch.org/docs/stable/torchvision/transforms.html#transforms-on-pil-image-and-torch-tensor) **et choisissez une ou des transformations applicables aux images de MNIST.**

In [ ]:
preprocess_train = transforms.Compose([
 transforms.Grayscale(num_output_channels=3), 
 transforms.Resize(224), 
 ### à compléter
 transforms.ToTensor(),              
 transforms.Normalize(               
 mean=[0.485, 0.456, 0.406],         
 std=[0.229, 0.224, 0.225]           
 )])

preprocess_test = transforms.Compose([
 transforms.Grayscale(num_output_channels=3), 
 transforms.Resize(224),         
 transforms.ToTensor(),              
 transforms.Normalize(               
 mean=[0.485, 0.456, 0.406],         
 std=[0.229, 0.224, 0.225]           
 )])

from torchvision import datasets
mnist_train = datasets.MNIST('./MNIST/', train=True, download=True,
                       transform=preprocess_train)
mnist_test = datasets.MNIST('./MNIST/', train=False, download=True,
                       transform=preprocess_test)

PyTorch a un objet `DataLoader` qui se charge de transférer les données par batch au modèle, avec shuffling aléatoire :

In [ ]:
batch_size = 1000
loader_train = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=4)
loader_test = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=True, num_workers=4)

*Création de l'optimizer*

En suivant l'exemple du TD2, créer un optimizer pour votre modèle. Rappelez-vous que cet optimizer ne doit mettre à jour que les paramètres de la dernière couche du AlexNet.

In [ ]:
# liste des paramètres à mettre à jour
params_to_update = []
### à compléter

learning_rate = 0.001
momentum = 0.9
### à compléter : création de l'optimizer

Solution plus bas

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

![](http://eurinfac.com/wp-content/uploads/2015/02/image-blanche.png)

Solution :

In [ ]:
# liste des paramètres à mettre à jour
params_to_update = []
for param in model.parameters():
  if param.requires_grad == True:
    params_to_update.append(param)

### création de l'optimizer
learning_rate = 0.001
momentum = 0.9
optimizer = torch.optim.SGD(params_to_update, lr=learning_rate, momentum=momentum)

*Entrainement du réseau*

Nous allons procéder à l'entrainement du réseau avec une fonction de coût adaptée à la classification. Essayons par exemple `torch.nn.CrossEntropyLoss`.

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

Entrainons la dernière couche du réseau pendant une époque :

In [ ]:
model.train() # on se remet en mode 'entrainement'

if torch.cuda.is_available():
    model.to('cuda')

for epoch in range(1):  # loop over the dataset multiple times

    for i, data in enumerate(loader_train, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        if torch.cuda.is_available():
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, loss.item()))

print('Finished Training')

Testons maintenant notre nouveau réseau sur les données de test :

In [ ]:
model.eval()

average_loss = 0.0

for i, data in enumerate(loader_test, 0):
  # get the inputs; data is a list of [inputs, labels]
  inputs, labels = data

  if torch.cuda.is_available():
    inputs = inputs.to('cuda')
    labels = labels.to('cuda')

  # forward
  outputs = model(inputs)
  loss = loss_function(outputs, labels)

  average_loss += loss.item()
  
  # print statistics
  print('%5d loss: %.3f' %
    (i + 1, loss.item()))

print('Finished testing. Average loss: %.3f' %
      (average_loss/(i+1)))

Voyons aussi ce que fait le réseau sur quelques images prises au hasard :

In [ ]:
inputs, classes = next(iter(loader_test))

plt.imshow(  inputs[0].permute(1, 2, 0)  )
print('classe prédite %d : ' %
      classes[0].item())

In [ ]:
if torch.cuda.is_available():
    inputs = inputs.to('cuda')
    labels = labels.to('cuda')

batch_t = torch.unsqueeze(inputs[0], 0)
output = model(batch_t)

score, index = torch.max(output, 1)
print(index.item(), score.item())

scores, indices = torch.sort(output, descending=True)
top3 = [(ind.item(), sc.item()) for sc, ind in zip(scores[0][:3], indices[0][:3])]
print(top3)

Félicitations, vous avez entrainé votre premier réseau de neurones ! Dans le prochain TD nous examinerons son fonctionnement en détail grâce à la visualisation.